С онтологиями (хранящимися в виде файлов) также можно работать при помощи Питона.<br>
Для этого существует несколько специализированных библиотек ([ontospy](https://pypi.org/project/ontospy/), [neo4j](https://github.com/neo4j-examples/movies-python-py2neo), [pronto](https://github.com/althonos/pronto), [RDFlib](https://rdflib.readthedocs.io/en/stable/) и др)<br>
Сегодня мы поговорим про *owlready* ([вот ее документация](https://pythonhosted.org/Owlready2/intro.html))

Что она умеет?<br>
- импортировать онтологии в формате RDF/XML, OWL/XML и NTriples.
- работать с классами и экземплярами онтологий как с питонскими объектами
- добавлять методы к классам онтологий
- автоматически классифицировать экземпляры с помощью HermiT ([здесь](http://www.hermit-reasoner.com/) о нем можно почитать больше)


In [ ]:
# скачиваем библиотеку
! pip3 install owlready2

     |████████████████████████████████| 20.9MB 1.4MB/s 
  Created wheel for owlready2: filename=Owlready2-0.25-cp36-cp36m-linux_x86_64.whl size=20269671 sha256=44e081cb28dbd607de1a1b890994c9aba70c8e55b5d545391e39caa0d07ff9c0
  Stored in directory: /root/.cache/pip/wheels/62/74/7f/ab814059f816d87b26a1e1ec242c40d443cd2e22b0868a2a94
Successfully built owlready2


In [ ]:
# импортируем библиотеку и все из нее
from owlready2 import *

**Создание онтологии**

Пустая онтология создается с помощью функции ```get_ontology()```<br>

Ее аргумент - это IRI онтологии.
([IRI](https://ru.wikipedia.org/wiki/Internationalized_Resource_Identifier) - это идентификатор онтологии, аналог привычного нам URL)

In [ ]:
 onto = get_ontology("http://test.org/onto.owl")

Пустую онтологию можно наполнять классами, а классы - объектами

Однако иногда случается работать с уже существующей онтологией, хранящейся в виде файла. В таком случае необходимо ее загрузить. <br>

**Загрузка онтологии**
Нам потребуется метод ```.load() ```

Онтологию можно загрузить из локальной директории или с сайта. 

In [ ]:
# создаем переменную onto, в которой сохранится онтология, взятая по заданному адресу

onto = get_ontology("http://www.lesfleursdunormal.fr/static/_downloads/pizza_onto.owl").load()

Хорошо, мы загрузили онтологию. Теперь можно посмотреть на ее содержимое

Содержимое можно получить используя dot notation (.), как в классическом ООП ([здесь recap про ООП](https://www.datacamp.com/community/tutorials/python-oop-tutorial)). Так можно получить доступ к классам, экземплярам, их свойствам, свойствам аннотации и др

In [ ]:
onto.base_iri # через точку мы получаем доступ к атрибуту онтологии (в данном случае это IRI)

'http://www.lesfleursdunormal.fr/static/_downloads/pizza_onto.owl#'

Посмотрим на классы онтологии:

In [ ]:
# обычный метод .classes() будет возвращать генератор с классами. Обернем это в функцию list() чтобы увидеть результат за один рантайм
list(onto.classes()) 

[pizza_onto.CheeseTopping,
 pizza_onto.FishTopping,
 pizza_onto.MeatTopping,
 pizza_onto.Pizza,
 pizza_onto.TomatoTopping,
 pizza_onto.Topping]

у нашей онтологии есть классы Топпингов, и Пицца

In [ ]:
print(onto.Pizza) # посмотрим на класс Пицца

pizza_onto.Pizza


синтаксис с квадратными скобками также разрешен (делает то же самое что и dot notation)

In [ ]:
print(onto["Topping"]) 

pizza_onto.Topping


In [ ]:
# попробуем получить доступ к несуществующему классу
print(onto["Dough_type"]) 

None


**к чему можно получить доступ:**
    

```
.classes()
.individuals()
.object_properties()
.data_properties()
.annotation_properties() 
.properties() 
.disjoint_classes()
.disjoint_properties()
.different_individuals() 
.get_namepace(base_iri)

 и др
```




**поиск в онтологии**

осуществляется с помощью метода ```.search()```<br>

что можно искать:

- iri (идентификатор онтологии)
- type (для экземпляров класса)
- subclass_of (подклассы класса)
- is_a (поиск экземпляров и подклассов класса)
- типы связей
- лейблы

In [ ]:
onto.search(iri = "*Topping") # * обозначает "любой класс, заканчивающийся на Topping"

[pizza_onto.CheeseTopping, pizza_onto.FishTopping, pizza_onto.MeatTopping, pizza_onto.TomatoTopping, pizza_onto.Topping]

**Сохранение онтологии**<br>
метод .save() сохранит онтологию в локальную директорию. 
Файлу можно задать имя и формат


In [ ]:
onto.save(file = "Pizza_onto_template", format = "rdfxml")

In [ ]:
!ls # проверим, что файл сохранился

Pizza_onto_template  sample_data


**редактирование онтологии**

В пицца-онтологии создадим новый класс ```NonVegetarianPizza```. Он наследуется от класса Pizza (то есть обладает всеми его свойствами и методами), но также может иметь свои уникальные <br>





In [ ]:
class NonVegetarianPizza(onto.Pizza):
    equivalent_to = [
                     onto.Pizza 
                     & ( onto.has_topping.some(onto.MeatTopping)
                     | onto.has_topping.some(onto.FishTopping)
                     ) ]

    def eat(self): print("Wait! I'm vegetarian!")
    def contents(self): print(list(onto.classes()))

In [ ]:
# создадим новый объект класса Pizza, сохраним в переменную test_pizza
test_pizza = onto.Pizza("test_pizza_owl_identifier")

In [ ]:
# добавим топпинги
test_pizza.has_topping = [ onto.CheeseTopping(),
                         onto.TomatoTopping() ]

In [ ]:
# и еще один топпинг, новым способом: листы можно модифицировать мгновенно методом .append()
test_pizza.has_topping.append(onto.MeatTopping())

In [ ]:
# проверим принадлежность объекта к классу
test_pizza.__class__

pizza_onto.Pizza

In [ ]:

# Execute HermiT and reparent instances and classes
sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.6/dist-packages/owlready2/hermit:/usr/local/lib/python3.6/dist-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpab3xvy1p
* Owlready2 * HermiT took 1.6741433143615723 seconds
* Owlready * Reparenting pizza_onto.test_pizza_owl_identifier: {pizza_onto.Pizza} => {pizza_onto.NonVegetarianPizza}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [ ]:
# проверим принадлежность объекта к классу
test_pizza.__class__

pizza_onto.NonVegetarianPizza

In [ ]:
test_pizza.eat()

Wait! I'm vegetarian!


In [ ]:
test_pizza.contents()

[pizza_onto.CheeseTopping, pizza_onto.FishTopping, pizza_onto.MeatTopping, pizza_onto.Pizza, pizza_onto.TomatoTopping, pizza_onto.Topping, pizza_onto.NonVegetarianPizza]
